<a href="https://colab.research.google.com/github/OmerRosen/Kaggle/blob/main/IMDB_2022_Hit_or_Flop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB 2022 - Hit or Flop

## Mission Statement

### Data To Collect

#### Search box page:
* Movie Id
* Movie Title
* Movie year
* Rating
* MetaScore - Outcome
* Description
* Poster
* Directore name + Link
* Stars
* Votes - Output
* Genre

#### Main Movie Page data
* Movie Length
* Rank
* Writer
* Star #1
* Star #2
* Star #3
* User eviews
* Critic reviews
* Number of photos posted
* Storyline - Text
* Tag line
* Release Date - Month
* Country of origin
* Additional Lanaguages
* Country of filming
* Production companies
* Budget
* Opening Weekend Date
* Gross US & Canada - Output
* Opening weekend US & Canada - Output
* Gross worldwide - Output
* Color - Color
* Color - Black&White
* Sound mix - Dolby Digital
* Sound mix - Dolby Atmos
* Aspect ratio

#### Director Page:
* Director Age
* Is Top 500?
* Director Gender (Based on bio)
* Previous film count - As Director
* Previous film count - As Writer
* Previous film count - As As Producer
* Director Publicity listing count

#### Information Abount Cast:
* Full list of cast and their profile links
* Number of cast members
* Produced by - Is top 500?
* Music by - Is top 500?
* Cinematography by  - Is top 500
* Film Editing by   - Is top 500
* Art Direction by  - Is top 500
* Number of Production Management
* Number of Art Department
* Number of Sound Department
* Number of Camera and Electrical Department
* Number of Editorial Department
* Number of Music Department
* Number of Additional Crew

#### Actor Page
* Is Top 5000?
* Is Top 500?
* Is Top 100 (aka - Has numberical rank)
* Is Top 10 (aka - Has numberical rank)
* Numerical Rank (Could be none)
* Gender
* Age
* Oscar nominations 
* Birth country

#### Aggragated data:
* How many male stars
* How many female stars
* Avrage cast memeber age
* Max/Min age of cast member
* Number of unique birth countries of actors
* Total number of Oscar numinatior for cast
* Num cast members in top 5000
* Num cast memebers in top 500
* Num cast members in top 100
* Num cast members in top 10

# IMDB Scraper

## Import Libraries

In [1]:
import requests
from IPython.core.display import HTML
import json
from time import sleep
import re
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import traceback


## Set main variables

In [2]:
base_url = "https://www.imdb.com"
base_folder_path = "/content/drive/My Drive/Harvard HW/Course 4 - Final Project"

start_date = "2022-01-01"
end_date = "2022-12-31"
minimum_votes = 10 # Minimum vote amount to coolect movie
start_point = 1 # Start from movie #1-50

movie_search_url = f"{base_url}/search/title/?title_type=feature&release_date={start_date},{end_date}&num_votes={minimum_votes},&{start_point}anguages=en&start=1&ref_=adv_nxt"
movie_search_url

'https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=10,&1anguages=en&start=1&ref_=adv_nxt'

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

list_of_50_movies = requests.get(f'{movie_search_url}', headers=headers, timeout=10)
list_of_50_movies

<Response [200]>

In [4]:
list_of_50_movies_soup = BeautifulSoup(list_of_50_movies.text, 'html.parser').find_all('div',{'class':'lister-item mode-advanced'})


In [5]:
movie_search_page = list_of_50_movies_soup[0]

## Main Functions

### Get Movie Box details

In [6]:
def get_basic_details(movie_search_page):
  search_box_info = {}
  search_box_info['movie_name'] = movie_search_page.find('a')
  search_box_info['movie_page_url'] = f"{base_url}{movie_search_page.find('a')['href']}"

  try:
    search_box_info['movie_place'] = None if not movie_search_page.find('span',{'class':'lister-item-index unbold text-primary'}) else movie_search_page.find('span',{'class':'lister-item-index unbold text-primary'}).text.replace('.','')
    search_box_info['movie_id'] = None if not search_box_info['movie_page_url'] else search_box_info['movie_page_url'].split('/')[-2]
    search_box_info['movie_name'] = None if not movie_search_page.find('a').find('img') else movie_search_page.find('a').find('img')['alt'].strip()
    search_box_info['movie_thubmnail'] = None if not movie_search_page.find('a').find('img') else movie_search_page.find('a').find('img')['src']
    #search_box_info['movie_year'] = None if not movie_search_page.find('span',{'class':'lister-item-year text-muted unbold'}) else movie_search_page.find('span',{'class':'lister-item-year text-muted unbold'}).text.replace('(','').replace(')','')
    #search_box_info['movie_rating'] = None if not movie_search_page.find('strong') else movie_search_page.find('strong').text
    search_box_info['movie_metascore'] = None if not movie_search_page.find('span', {'class': 'metascore'}) else movie_search_page.find('span', {'class': 'metascore'}).text.strip()
    search_box_info['movie_description'] = None if not movie_search_page.find_all('p',{'class':'text-muted'}) else movie_search_page.find_all('p',{'class':'text-muted'})[1].text.strip()
    search_box_info['runtime_min'] = None if not movie_search_page.find('span',{'class':'runtime'}) else movie_search_page.find('span',{'class':'runtime'}).text.split(' ')[0]

    bottom_box_info = movie_search_page.find('p',{'class':'sort-num_votes-visible'}).find_all('span')
    search_box_info['movie_vote_num'] = bottom_box_info[1]['data-value']
    search_box_info['movie_gross'] = bottom_box_info[4]['data-value'] if len(bottom_box_info)>2 else None;

    search_box_info['movie_rating'] = None if not movie_search_page.find_all('p',{'class':'text-muted'})[0].find('span',{'class':'certificate'}) else movie_search_page.find_all('p',{'class':'text-muted'})[0].find('span',{'class':'certificate'}).text.strip()
    search_box_info['movie_genere'] = None if not movie_search_page.find_all('p',{'class':'text-muted'})[0].find('span',{'class':'genre'}) else movie_search_page.find_all('p',{'class':'text-muted'})[0].find('span',{'class':'genre'}).text.strip()

    search_box_info['__SuccsefullyCollectBasicDetails'] = True;

  except Exception as e:
    print(f"Failed extracting data for movie: {search_box_info['movie_name']}. \nUrl: {search_box_info['movie_page_url']}.\n Error:\n{e}")
    traceback.print_exc()
    search_box_info['__SuccsefullyCollectBasicDetails'] = False;


  return search_box_info

In [7]:
movie_search_page = list_of_50_movies_soup[1]
search_box_info = get_basic_details(movie_search_page)
search_box_info



{'movie_name': 'Everything Everywhere All at Once',
 'movie_page_url': 'https://www.imdb.com/title/tt6710474/?ref_=adv_li_i',
 'movie_place': '2',
 'movie_id': 'tt6710474',
 'movie_thubmnail': 'https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png',
 'movie_metascore': '81',
 'movie_description': 'A middle-aged Chinese immigrant is swept up into an insane adventure in which she alone can save existence by exploring other universes and connecting with the lives she could have led.',
 'runtime_min': '139',
 'movie_vote_num': '414055',
 'movie_gross': '72,861,480',
 'movie_rating': 'R',
 'movie_genere': 'Action, Adventure, Comedy',
 '__SuccsefullyCollectBasicDetails': True}

### Get Movie Page Details

In [8]:
movie_main_page = requests.get(search_box_info['movie_page_url'], headers=headers)
movie_main_page_soup = BeautifulSoup(movie_main_page.text, 'html.parser')

In [9]:
currency_codes = {
    "$":"USD",
    "€":"EUR",
    "£":"GBP",
    "₹":"INR",
    "â‚¬": "EUR",
    "â‚¹": "INR",
    "CA$": "CAD",
    "NOKÂ": "NOK",
    "Â£": "GBP",
    "CHF": "CHF",
    "Â¥": "JPY",
    "PKR": "PKR",
    "A$": "AUD",
    "CZK": "CZK",
    "RUR": "RUB",
    "NZ$": "NZD",
    "MYR": "MYR",
    "NGN": "NGN"
}


currency_conversion_values = {}


In [10]:
def extract_currency_symbol(s):
    match = re.search(r"[^\d]+", s)
    if match:
        return match.group(0).strip()
    else:
        return ""

In [11]:
def get_currency_code(amount_string):
  currency_symbol = extract_currency_symbol(amount_string)
  if currency_symbol in currency_codes:
      return currency_codes[currency_symbol]
  else:
    print(f"Could not find a value for: {currency_symbol} in {amount_string}")
    return None

In [12]:
def extract_numerical_value(string):
    # Remove all non-numeric characters from the string
    numerical_string = re.sub(r"[^\d.]+", "", string)
    # Convert the string to a float and return it
    return float(numerical_string)

In [13]:

with open(f'{base_folder_path}/openexchangerates.txt', 'r') as f:
    api_key = f.read().strip()

api_key

def convert_string_amount_to_usd(amount):

  # get the currency code and amount value
  currency_code = get_currency_code(amount)
  original_amount = extract_numerical_value(amount)
  usd_amount = None

  if currency_code==None:
    usd_amount=None
  elif currency_code!="USD":
  
    if currency_code in currency_conversion_values:
      exchange_rate = currency_conversion_values[currency_code]
    else:
      # make API request to get exchange rate for the currency code
      url = f"https://openexchangerates.org/api/latest.json?app_id={api_key}&symbols={currency_code}"
      print(amount,currency_code,original_amount,url)
      response = requests.get(url)
      print(response)
      # parse the exchange rate from the API response
      exchange_rate = response.json()["rates"][currency_code]
      
      currency_conversion_values[currency_code] = exchange_rate
      
      # calculate the USD equivalent amount
      usd_amount = round(original_amount / exchange_rate)
      
      print(f"{amount} {currency_code} = {usd_amount} USD")
  else:
    usd_amount = original_amount

  return usd_amount,original_amount,currency_code



convert_string_amount_to_usd("CA$15,000")
convert_string_amount_to_usd("NOKÂ 80,200,000")
convert_string_amount_to_usd("NOKÂ 20,000,000")
convert_string_amount_to_usd("â‚¹3,500,000,000")
currency_conversion_values

CA$15,000 CAD 15000.0 https://openexchangerates.org/api/latest.json?app_id=ac63294868f6455a91a128e22f4d9f35&symbols=CAD
<Response [200]>
CA$15,000 CAD = 11163 USD
NOKÂ 80,200,000 NOK 80200000.0 https://openexchangerates.org/api/latest.json?app_id=ac63294868f6455a91a128e22f4d9f35&symbols=NOK
<Response [200]>
NOKÂ 80,200,000 NOK = 7785940 USD
â‚¹3,500,000,000 INR 3500000000.0 https://openexchangerates.org/api/latest.json?app_id=ac63294868f6455a91a128e22f4d9f35&symbols=INR
<Response [200]>
â‚¹3,500,000,000 INR = 42572305 USD


{'CAD': 1.343698, 'NOK': 10.300619, 'INR': 82.213073}

In [14]:
def get_extended_details(movie_page_url, movie_name):
  movie_page_dict = {}

  try:

    movie_main_page = requests.get(movie_page_url, headers=headers)
    movie_main_page_soup = BeautifulSoup(movie_main_page.text, 'html.parser')   

    year_rating = movie_main_page_soup.find('ul',{"class":"ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt"}).find_all('a')

    movie_page_dict['movie_year'] = year_rating[0].text if len(year_rating)>0 else None
    movie_page_dict['movie_rating'] = year_rating[1].text if len(year_rating)>1 else None

    director_box = movie_main_page_soup.find('span',string="Director") if movie_main_page_soup.find('span',string="Directors") is None else movie_main_page_soup.find('span',string="Directors")
    if director_box == None:
      director_box = movie_main_page_soup.find('a',string="Director") if movie_main_page_soup.find('span',string="Directors") is None else movie_main_page_soup.find('a',string="Directors")
    director_box = director_box.find_next_sibling()
    movie_page_dict['director_name'] = director_box.find('a').text
    movie_page_dict['director_url'] = f"{base_url}{director_box.find('a')['href']}"
    movie_page_dict['director_id'] = movie_page_dict['director_url'].split('/')[-2]

    writers_box = movie_main_page_soup.find('span',string="Writer") if movie_main_page_soup.find('span',string="Writers") is None else movie_main_page_soup.find('span',string="Writers")
    if writers_box == None:
      writers_box = movie_main_page_soup.find('span',string="Writer") if movie_main_page_soup.find('a',string="Writers") is None else movie_main_page_soup.find('a',string="Writers")
    # If not writer specified - Assume director is also writer
    writers_box = writers_box.find_next_sibling() if writers_box!=None else director_box
    movie_page_dict['writer_name'] = writers_box.find('a').text
    movie_page_dict['writer_url'] = f"{base_url}{writers_box.find('a')['href']}"
    movie_page_dict['writer_id'] = movie_page_dict['writer_url'].split('/')[-2]

    movie_page_dict['movie_cast_url'] = movie_main_page_soup.find('a',{'class':'ipc-metadata-list-item__label ipc-metadata-list-item__label--link'})['href']
    movie_page_dict['movie_cast_url'] = f"{base_url}{movie_page_dict['movie_cast_url']}"
    stars_box = movie_main_page_soup.find('a',{'class':'ipc-metadata-list-item__label ipc-metadata-list-item__label--link'}).find_parent().find_all('a')
    for i,star_box in enumerate(stars_box[1:-1]):
      star_num = f'star_{i+1}'
      movie_page_dict[f'{star_num}_name'] = star_box.text.strip()
      movie_page_dict[f'{star_num}_url'] = f"{base_url}{star_box['href']}"
      movie_page_dict[f'{star_num}_id'] = star_box['href'].split('/')[-2]

    review_scores = movie_main_page_soup.find_all('span',{'class':'score'})
    movie_page_dict['user_reviews_count'] = review_scores[0].text if len(review_scores)>0 else None
    movie_page_dict['critic_reviews_count'] = review_scores[1].text if len(review_scores)>1 else None

    movie_page_dict['release_date'] = movie_main_page_soup.find('a',{'class':'ipc-metadata-list-item__label ipc-metadata-list-item__label--link'}, string="Release date").find_parent().find('a',{'class':'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'}).text.split('(')[0].strip()
    movie_page_dict['release_date']

    movie_page_dict['budget'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Budget")
    if (movie_page_dict['budget']!=None):
      movie_page_dict['budget'] = movie_page_dict['budget'].find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()

      usd_amount,original_amount,currency_code = convert_string_amount_to_usd(movie_page_dict['budget'])
      movie_page_dict['budget_usd'] = usd_amount
      movie_page_dict['budget_currency'] = currency_code

    movie_page_dict['gross_worldwide'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Gross worldwide")
    if (movie_page_dict['gross_worldwide']!=None):
      movie_page_dict['gross_worldwide'] = movie_page_dict['gross_worldwide'].find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()

    movie_page_dict['gross_us_canada'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Gross US & Canada")
    if (movie_page_dict['gross_us_canada']!=None):
      movie_page_dict['gross_us_canada'] = movie_page_dict['gross_us_canada'].find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()

    movie_page_dict['opening_weekend_us_canada'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Opening weekend US & Canada")
    if (movie_page_dict['opening_weekend_us_canada']!=None):
      movie_page_dict['opening_weekend_us_canada'] = movie_page_dict['opening_weekend_us_canada'].find_next_sibling().find('span',{'class':'ipc-metadata-list-item__list-content-item'}).text.split('(')[0].strip()

    movie_page_dict['origin_country'] = movie_main_page_soup.find('span',{'class':'ipc-metadata-list-item__label'}, string="Country of origin")
    if (movie_page_dict['origin_country']!=None):
      movie_page_dict['origin_country'] = movie_page_dict['origin_country'].find_next_sibling().find('a').text.split('(')[0].strip()

    languages = movie_main_page_soup.find('span',string="Language") if movie_main_page_soup.find('span',string="Languages") is None else movie_main_page_soup.find('span',string="Languages")
    languages = [] if not languages else languages.find_next_sibling().find_all('a')
    movie_page_dict['languages'] = ",".join([language.text for language in languages])

    movie_page_dict['__SuccsefullyCollectExtandedDetails'] = True;

    

  except Exception as e:
    print(f"Failed extracting data for movie: {movie_name}. \nUrl: {movie_page_url}.\n Error:\n{e}")
    traceback.print_exc()
    movie_page_dict['__SuccsefullyCollectExtandedDetails'] = False;

  return movie_page_dict


get_extended_details(search_box_info['movie_page_url'],search_box_info['movie_name'])

{'movie_year': '2022',
 'movie_rating': 'R',
 'director_name': 'Daniel Kwan',
 'director_url': 'https://www.imdb.com/name/nm3453283/?ref_=tt_ov_dr',
 'director_id': 'nm3453283',
 'writer_name': 'Daniel Kwan',
 'writer_url': 'https://www.imdb.com/name/nm3453283/?ref_=tt_ov_wr',
 'writer_id': 'nm3453283',
 'movie_cast_url': 'https://www.imdb.com/title/tt6710474/fullcredits/cast?ref_=tt_ov_st_sm',
 'star_1_name': 'Michelle Yeoh',
 'star_1_url': 'https://www.imdb.com/name/nm0000706/?ref_=tt_ov_st',
 'star_1_id': 'nm0000706',
 'star_2_name': 'Stephanie Hsu',
 'star_2_url': 'https://www.imdb.com/name/nm3513533/?ref_=tt_ov_st',
 'star_2_id': 'nm3513533',
 'star_3_name': 'Jamie Lee Curtis',
 'star_3_url': 'https://www.imdb.com/name/nm0000130/?ref_=tt_ov_st',
 'star_3_id': 'nm0000130',
 'user_reviews_count': '3.3K',
 'critic_reviews_count': '395',
 'release_date': 'April 8, 2022',
 'budget': '$14,300,000',
 'budget_usd': 14300000.0,
 'budget_currency': 'USD',
 'gross_worldwide': '$135,728,167',

In [15]:
# print(search_box_info['movie_page_url'],search_box_info['movie_name'])
# cast_url = movie_main_page_soup.find('a',{'class':'ipc-metadata-list-item__label ipc-metadata-list-item__label--link'})['href']
# cast_url = f"{base_url}{cast_url}"
# cast_url

## Cursor for collecting movies in batches

In [16]:
# Fixed parameters:
base_url = "https://www.imdb.com"

start_date = "2022-01-01"
end_date = "2022-12-31"
minimum_votes = 20 # Minimum vote amount to coolect movie

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"}

In [17]:
# Dynamic parameters:

start_point = 1 # Start from movie #1-50

movie_search_url_50_batch = f"{base_url}/search/title/?title_type=feature&release_date={start_date},{end_date}&num_votes={minimum_votes},&user_rating=1.0,10.0&countries=us&languages=en&start={start_point}&ref_=adv_nxt"
print(f"movie_search_url_50_batch: {movie_search_url_50_batch}")
search_page = requests.get(f'{movie_search_url_50_batch}', headers=headers, timeout=10)
search_page_soup = BeautifulSoup(list_of_50_movies.text, 'html.parser')
list_of_50_movies_soup = search_page_soup.find_all('div',{'class':'lister-item mode-advanced'})

max_num_of_results = search_page_soup.find('div',{'class':'desc'}).find('span').text.split(' ')[2].replace(",", "")
max_num_of_results = int(max_num_of_results)

print(f"max_num_of_results: {max_num_of_results}")

movie_search_url_50_batch: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&user_rating=1.0,10.0&countries=us&languages=en&start=1&ref_=adv_nxt
max_num_of_results: 6302


In [18]:
search_page = requests.get(f'{movie_search_url_50_batch}', headers=headers, timeout=10)
search_page_soup = BeautifulSoup(search_page.text, 'html.parser')
list_of_50_movies_soup = search_page_soup.find_all('div',{'class':'lister-item mode-advanced'})

max_num_of_results = search_page_soup.find('div',{'class':'desc'}).find('span').text.split(' ')[2].replace(",", "")
max_num_of_results = int(max_num_of_results)
max_num_of_results

1383

In [19]:
total_movie_dataset = []
total_actor_dataset = []
total_director_dataset = []

start_point = 1 # Start from movie #1-50

while start_point < max_num_of_results-50:

  movie_search_url_50_batch = f"{base_url}/search/title/?title_type=feature&release_date={start_date},{end_date}&num_votes={minimum_votes},&languages=en&start={start_point}&ref_=adv_nxt"
  print(f"movie_search_url_50_batch start point {start_point}: {movie_search_url_50_batch}")
  search_page = requests.get(f'{movie_search_url_50_batch}', headers=headers, timeout=10)
  search_page_soup = BeautifulSoup(search_page.text, 'html.parser')
  list_of_50_movies_soup = search_page_soup.find_all('div',{'class':'lister-item mode-advanced'})

  for i,movie in enumerate(list_of_50_movies_soup):
    movie_data = {}

    basic_data = get_basic_details(movie)
    movie_data.update(basic_data)

    if basic_data['__SuccsefullyCollectBasicDetails']==True:
      movie_main_page = requests.get(basic_data['movie_page_url'], headers=headers)
      movie_main_page_soup = BeautifulSoup(movie_main_page.text, 'html.parser')   

      if i%30==0:
        print(basic_data['movie_name'],basic_data['movie_page_url'])

      extended_deta = get_extended_details(basic_data['movie_page_url'],basic_data['movie_name'])
      movie_data.update(extended_deta)

      sleep(0.1)

      total_movie_dataset.append(movie_data)
    else:
      movie_data['__SuccsefullyCollectExtandedDetails'] = False;
      print(f"Skipping extanded data for: {basic_data['movie_name']} - {basic_data['movie_page_url']}")

  movie_dataset = pd.DataFrame(total_movie_dataset)
  movie_dataset.to_csv("/content/drive/My Drive/Harvard HW/Course 4 - Final Project/total_movie_dataset.csv", index=False)

  start_point += 50

movie_search_url_50_batch start point 1: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=1&ref_=adv_nxt
Avatar: The Way of Water https://www.imdb.com/title/tt1630029/?ref_=adv_li_i
€10,000,000 EUR 10000000.0 https://openexchangerates.org/api/latest.json?app_id=ac63294868f6455a91a128e22f4d9f35&symbols=EUR
<Response [200]>
€10,000,000 EUR = 10888597 USD
Could not find a value for: ₹ in ₹3,500,000,000
Women Talking https://www.imdb.com/title/tt13669038/?ref_=adv_li_i
movie_search_url_50_batch start point 51: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=51&ref_=adv_nxt
Terrifier 2 https://www.imdb.com/title/tt10403420/?ref_=adv_li_i
Roald Dahl's Matilda the Musical https://www.imdb.com/title/tt3447590/?ref_=adv_li_i
movie_search_url_50_batch start point 101: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-1

Traceback (most recent call last):
  File "<ipython-input-14-4f1d2746ad1e>", line 17, in get_extended_details
    director_box = director_box.find_next_sibling()
AttributeError: 'NoneType' object has no attribute 'find_next_sibling'


Could not find a value for: ₹ in ₹1,800,000,000
movie_search_url_50_batch start point 301: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=301&ref_=adv_nxt
Play Dead https://www.imdb.com/title/tt20198774/?ref_=adv_li_i
Could not find a value for: ₹ in ₹4,100,000,000
Maneater https://www.imdb.com/title/tt15262370/?ref_=adv_li_i
CHF 2,600,000 CHF 2600000.0 https://openexchangerates.org/api/latest.json?app_id=ac63294868f6455a91a128e22f4d9f35&symbols=CHF
<Response [200]>
CHF 2,600,000 CHF = 2845318 USD
movie_search_url_50_batch start point 351: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=351&ref_=adv_nxt
Somewhere in Queens https://www.imdb.com/title/tt14097180/?ref_=adv_li_i
Failed extracting data for movie: V/H/S/99. 
Url: https://www.imdb.com/title/tt21651560/?ref_=adv_li_i.
 Error:
'NoneType' object has no attribute 'find_next_sibling'


Traceback (most recent call last):
  File "<ipython-input-14-4f1d2746ad1e>", line 17, in get_extended_details
    director_box = director_box.find_next_sibling()
AttributeError: 'NoneType' object has no attribute 'find_next_sibling'


Seriously Red https://www.imdb.com/title/tt4586828/?ref_=adv_li_i
Could not find a value for: ¥ in ¥900,000,000
movie_search_url_50_batch start point 401: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=401&ref_=adv_nxt
Moloch https://www.imdb.com/title/tt14412446/?ref_=adv_li_i
Who Invited Them https://www.imdb.com/title/tt17677178/?ref_=adv_li_i
movie_search_url_50_batch start point 451: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=451&ref_=adv_nxt
The Requin https://www.imdb.com/title/tt11348232/?ref_=adv_li_i
Failed extracting data for movie: Marmaduke. 
Url: https://www.imdb.com/title/tt8746404/?ref_=adv_li_i.
 Error:
'NoneType' object has no attribute 'find_next_sibling'


...

movie_search_url_50_batch start point 1001: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=1001&ref_=adv_nxt
Low Life https://www.imdb.com/title/tt6165892/?ref_=adv_li_i
Failed extracting data for movie: Encanto Sing-Along. 
Url: https://www.imdb.com/title/tt19175754/?ref_=adv_li_i.
 Error:
'NoneType' object has no attribute 'find_next_sibling'


Traceback (most recent call last):
  File "<ipython-input-14-4f1d2746ad1e>", line 17, in get_extended_details
    director_box = director_box.find_next_sibling()
AttributeError: 'NoneType' object has no attribute 'find_next_sibling'


Student Seduction https://www.imdb.com/title/tt14947744/?ref_=adv_li_i
MYR 8,000,000 MYR 8000000.0 https://openexchangerates.org/api/latest.json?app_id=ac63294868f6455a91a128e22f4d9f35&symbols=MYR
<Response [200]>
MYR 8,000,000 MYR = 1815088 USD
movie_search_url_50_batch start point 1051: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=1051&ref_=adv_nxt
Vide Noir https://www.imdb.com/title/tt11124146/?ref_=adv_li_i
Moon Manor https://www.imdb.com/title/tt8946942/?ref_=adv_li_i
movie_search_url_50_batch start point 1101: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=1101&ref_=adv_nxt
Presence https://www.imdb.com/title/tt12993004/?ref_=adv_li_i
White Paradise https://www.imdb.com/title/tt19895588/?ref_=adv_li_i
movie_search_url_50_batch start point 1151: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-

Traceback (most recent call last):
  File "<ipython-input-14-4f1d2746ad1e>", line 17, in get_extended_details
    director_box = director_box.find_next_sibling()
AttributeError: 'NoneType' object has no attribute 'find_next_sibling'


The Macabre https://www.imdb.com/title/tt11295396/?ref_=adv_li_i
movie_search_url_50_batch start point 1251: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=1251&ref_=adv_nxt
Amerikatsi https://www.imdb.com/title/tt13616368/?ref_=adv_li_i
The Third Saturday in October https://www.imdb.com/title/tt15978042/?ref_=adv_li_i
movie_search_url_50_batch start point 1301: https://www.imdb.com/search/title/?title_type=feature&release_date=2022-01-01,2022-12-31&num_votes=20,&languages=en&start=1301&ref_=adv_nxt
Albatross https://www.imdb.com/title/tt12992082/?ref_=adv_li_i
Free Skate https://www.imdb.com/title/tt9587950/?ref_=adv_li_i


In [20]:
movie_dataset = pd.DataFrame(total_movie_dataset)
movie_dataset.to_csv("/content/drive/My Drive/Harvard HW/Course 4 - Final Project/total_movie_dataset.csv", index=False)

movie_dataset

,movie_name,movie_page_url,movie_place,movie_id,movie_thubmnail,movie_metascore,movie_description,runtime_min,movie_vote_num,movie_gross,...,release_date,budget,budget_usd,budget_currency,gross_worldwide,gross_us_canada,opening_weekend_us_canada,origin_country,languages,__SuccsefullyCollectExtandedDetails
0,Avatar: The Way of Water,https://www.imdb.com/title/tt1630029/?ref_=adv...,1,tt1630029,https://m.media-amazon.com/images/S/sash/4Fyxw...,67,Jake Sully lives with his newfound family form...,192,337209,"659,682,302",...,"December 16, 2022","$350,000,000",350000000.0,USD,"$2,312,242,473","$681,826,264","$134,100,226",United States,English,True
1,Everything Everywhere All at Once,https://www.imdb.com/title/tt6710474/?ref_=adv...,2,tt6710474,https://m.media-amazon.com/images/S/sash/4Fyxw...,81,A middle-aged Chinese immigrant is swept up in...,139,414055,"72,861,480",...,"April 8, 2022","$14,300,000",14300000.0,USD,"$135,728,167","$77,020,896","$501,305",United States,"English,Mandarin,Cantonese",True
2,The Whale,https://www.imdb.com/title/tt13833688/?ref_=ad...,3,tt13833688,https://m.media-amazon.com/images/S/sash/4Fyxw...,60,"A reclusive, morbidly obese English teacher at...",117,117996,None,...,"December 21, 2022","$10,000,000",10000000.0,USD,"$53,082,539","$17,443,137","$332,152",United States,English,True
3,Triangle of Sadness,https://www.imdb.com/title/tt7322224/?ref_=adv...,4,tt7322224,https://m.media-amazon.com/images/S/sash/4Fyxw...,63,A fashion model celebrity couple join an event...,147,117987,None,...,"September 18, 2022","€10,000,000",10888597.0,EUR,"$25,377,810","$4,608,096","$214,602",None,"English,Swedish,German,French,Greek,Tagalog,Akan",True
4,Top Gun: Maverick,https://www.imdb.com/title/tt1745960/?ref_=adv...,5,tt1745960,https://m.media-amazon.com/images/S/sash/4Fyxw...,78,"After thirty years, Maverick is still pushing ...",130,561267,"718,732,821",...,"May 27, 2022","$170,000,000",170000000.0,USD,"$1,493,491,858","$718,732,821","$126,707,459",United States,English,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,Single and Ready to Jingle,https://www.imdb.com/title/tt18298074/?ref_=ad...,"1,346",tt18298074,https://m.media-amazon.com/images/S/sash/4Fyxw...,None,"Emma Warner, as a SVP of a toy company, spends...",90,362,None,...,"December 11, 2022",None,NaN,NaN,None,None,None,None,English,True
1346,Last the Night,https://www.imdb.com/title/tt14263856/?ref_=ad...,"1,347",tt14263856,https://m.media-amazon.com/images/S/sash/4Fyxw...,None,"During the pandemic, a burned out teacher on t...",88,6447,None,...,"August 11, 2022",None,NaN,NaN,None,None,None,United States,English,True
1347,Runner,https://www.imdb.com/title/tt13111170/?ref_=ad...,"1,348",tt13111170,https://m.media-amazon.com/images/S/sash/4Fyxw...,None,An 18-year-old girl dealing with the sudden de...,76,51,None,...,"September 9, 2022",None,NaN,NaN,None,None,None,None,English,True
1348,Night of the Bastard,https://www.imdb.com/title/tt16420502/?ref_=ad...,"1,349",tt16420502,https://m.media-amazon.com/images/S/sash/4Fyxw...,None,After an injured young woman takes refuge in h...,82,117,None,...,2022,None,NaN,NaN,None,None,None,United States,English,True


In [21]:
languages = movie_main_page_soup.find('span',string="Language") if movie_main_page_soup.find('span',string="Languages") is None else movie_main_page_soup.find('span',string="Languages")
languages = [] if not languages else languages.find_next_sibling().find_all('a')
",".join([language.text for language in languages])

'English'

In [22]:
# movie_main_page_soup
# writers_box = movie_main_page_soup.find('span',string="Writer") if movie_main_page_soup.find('a',string="Writers") is None else movie_main_page_soup.find('a',string="Writers")
# writers_box = writers_box.find_next_sibling()
# writers_box